In [2]:
!pip install keras-tuner

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch




In [6]:
#definicja modelu za pomocą funkcji
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers',min_value=1,max_value=3)):
    model.add(layers.Flatten(input_shape=(28,28)))
    model.add(layers.Dense(units=hp.Int('units'+str(i),min_value=32,max_value=512,step=32),
                           activation='relu'))
  model.add(layers.Dense(10,activation='softmax'))
  model.compile(optimizer=keras.optimizers.Adam(
      hp.Choice('learning_rate',
                values = [1e-2,1e-3,1e-4])),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [9]:
#inicjalizacja strojenia hiperparametrów

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial = 3,
    directory = 'my_dir',
    project_name='myparams'
)

In [10]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0

11490434/11490434 [==============================] - 0s 0us/step


In [11]:
tuner.search(x_train,y_train,
             epochs=5,
             validation_data=(x_test,y_test))

Trial 5 Complete [00h 07m 11s]
val_accuracy: 0.9769333402315775

Best val_accuracy So Far: 0.9793000022570292
Total elapsed time: 00h 19m 33s


In [13]:
best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()

Results summary
Results in my_dir/myparams
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 2
units0: 224
learning_rate: 0.001
units1: 288
Score: 0.9793000022570292

Trial 1 summary
Hyperparameters:
num_layers: 2
units0: 416
learning_rate: 0.001
units1: 32
Score: 0.9789333343505859

Trial 4 summary
Hyperparameters:
num_layers: 3
units0: 512
learning_rate: 0.0001
units1: 384
units2: 384
Score: 0.9769333402315775

Trial 3 summary
Hyperparameters:
num_layers: 3
units0: 256
learning_rate: 0.01
units1: 192
units2: 32
Score: 0.9611000021298727

Trial 0 summary
Hyperparameters:
num_layers: 1
units0: 32
learning_rate: 0.0001
Score: 0.9284999966621399


In [14]:
eval_loss,eval_accuracy = best_model.evaluate(x_test,y_test)
print(f'nalepszy model -> Loss: {eval_loss}, Accuracy: {eval_accuracy}')

313/313 [==============================] - 1s 2ms/step - loss: 0.0657 - accuracy: 0.9814
nalepszy model -> Loss: 0.06565467268228531, Accuracy: 0.9814000129699707
